<h1><center>Data Analysis - Astronomy</center></h1>

<center><b>Mean_datasets</b> function that reads in a list of CSV files and returns an array of the mean of each cell in the data files.</center>

The files each contain n rows and m columns, giving a total of n x m cells. The individual cells are separated by commas, and all CSV files in the list will have the same number of rows and columns.

In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def mean_datasets(arr):
    ar = []; 
    for i in range(len(arr)):
        ar.append(np.loadtxt(arr[i],delimiter = ','))
    ar = np.asarray(ar,dtype = float)
    m = np.zeros(shape = (ar[0].shape))
    cnt = 0
    for i in range(len(arr)):
        m = m + ar[i]
        cnt += 1 
    m = np.round(m/cnt,1)
    return m

### Flexible Image Transport
One of the most widely used formats for astronomical images is the <b>Flexible Image Transport System</b>. In a <a href = 'https://en.wikipedia.org/wiki/FITS'>FITS</a> file, the image is stored in a numerical array, which we can load into a NumPy array.

<p>FITS files also have headers which store metadata about the image.

FITS files are a standard format and astronomers have developed many libraries (in many programming languages) that can read and write FITS files. We're going to use the Astropy module.</p>

In [ ]:
hdulist = fits.open('/home/Tesla/Programming/Data_Astro/fits_images_all/image1.fits')
data = hdulist[0].data

#Visualising the data
plt.imshow(data,cmap=plt.cm.viridis)
plt.colorbar()
plt.show()

<h3> Finding the index of the maximum value in the image array</h3>

In [ ]:
def load_fits(file):
    hdulist = fits.open(file)
    data = hdulist[0].data
    index = np.unravel_index(data.argmax(), data.shape)
    return index

<h3>Stacking various fits images to reduce the signal noise ratio and detect a pulsar</h3>

In [ ]:
def mean_fits(arr_fits):
    ar = []
    for i in range(len(arr_fits)):
        ar.append(fits.open(arr_fits[i])[0].data)
    m = np.zeros(shape = (ar[0].shape))
    cnt = 0
    for i in range(len(arr_fits)):
        m += ar[i]; cnt += 1
    m /= cnt
    return m

### Equitorial coordinates
Equatorial coordinates are fixed relative to the celestial sphere, so the positions are independent of when or where the observations took place. They are defined relative to the celestial equator (which is in the same plane as the Earth's equator) and the ecliptic (the path the sun traces throughout the year).

A point on the celestial sphere is given by two coordinates:

  <p><center><b>Right ascension</b>: the angle from the vernal equinox to the point, going east along the celestial equator<br>
   <b>Declination</b>: the angle from the celestial equator to the point, going north (negative values indicate going south)</center></p>

The vernal equinox is the intersection of the celestial equator and the ecliptic where the ecliptic rises above the celestial equator going further east. 

In [ ]:
def hms2dec(hrs,mn,s):
    return (15*(hrs+mn/60+s/(60*60)))
def dms2dec(deg,am,ars):
    if deg > 0:
        return(deg + am/60 + ars/(60*60))
    else:
        return (-1*(abs(deg) + am/60 + ars/(60*60)))

### Angular Distance
To crossmatch two catalogues we need to compare the angular distance between objects on the celestial sphere.

People loosely call this a "distance", but technically its an angular distance: the projected angle between objects as seen from Earth.

If we have an object on the celestial sphere with right ascension and declination ($\alpha_1, \delta_1$), then the angular distance to another object with coordinates ($\alpha_2, \delta_2$) is: $$d = 2 \arcsin \sqrt{ \sin^2 \frac{|\delta_1 - \delta_2|}{2} + \cos \delta_1 \cos \delta_2 \sin^2 \frac{|\alpha_1 - \alpha_2|}{2} }$$

Angular distances have the same units as angles (degrees). There are other equations for calculating the angular distance but this one, called the <a href = 'https://en.wikipedia.org/wiki/Haversine_formula'>Haversine formula</a>, is good at avoiding floating point errors when the two points are close together. 

In [ ]:
import numpy as np

# Write your angular_dist function here.
def angular_dist(ra1,dec1,ra2,dec2):
    ra1 = np.radians(ra1); dec1 = np.radians(dec1);
    ra2 = np.radians(ra2); dec2 = np.radians(dec2)
    a = np.sin(abs(dec2-dec1)/2); b = np.sin(abs(ra1-ra2)/2)
    d = 2*np.arcsin(np.sqrt(a**2 + np.cos(dec1)*np.cos(dec2)*b**2))
    d = np.degrees(d)
    return d

### Format of Astronomical Catalogue - Understanding the format



Before we can crossmatch our two catalogues we first have to import the raw data. Every astronomy catalogue tends to have its own unique format so we'll need to look at how to do this with each one individually.

We'll look at the AT20G bright source sample survey first. The raw data we'll be using is the file table2.dat from <a  href = 'http://cdsarc.u-strasbg.fr/viz-bin/Cat?J/MNRAS/384/775'>this page</a> in the VizieR archives, but we'll use the filename bss.dat from now on.

Every catalogue in VizieR has a detailed README file that gives you the exact format of each table in the catalogue.


### AT20G BSS catalogue - Catalogue of bright radio sources

The catalogue is organised in fixed-width columns, with the format of the columns being:

    1: Object catalogue ID number (sometimes with an asterisk)
    2-4: Right ascension in HMS notation
    5-7: Declination in DMS notation
    8-: Other information, including spectral intensities
### SuperCOSMOS all-sky catalogue

The SuperCOSMOS all-sky catalogue is a catalogue of galaxies generated from several visible light surveys. 
The catalogue uses a comma-separated value (CSV) format. Aside from the first row, which contains column labels, the format is:

    1: Right ascension in decimal degrees
    2: Declination in decimal degrees
    3: Other data, including magnitude and apparent shape


In [ ]:
def import_bss():
    bss_data = []
    lst = np.loadtxt('Data/bss.dat', usecols = range(1,7))
    for i in range(len(lst)):
        bss_data.append((i+1,hms2dec(lst[i][0],lst[i][1],lst[i][2]),dms2dec(lst[i][3],lst[i][4],lst[i][5])))
    return bss_data

def import_super():
    super_data = []
    lst = np.loadtxt('Data/super.csv', delimiter = ',', skiprows = 1, usecols=[0,1])
    for i in range(len(lst)):
        super_data.append((i+1,lst[i][0],lst[i][1]))
    return super_data

In [ ]:
def find_closest(ar,ra,dec):
    ang_dist = []
    for i in range(len(ar)):
        ang_dist.append((ar[i][0],angular_dist(ar[i][1],ar[i][2],ra,dec)))
    return min(ang_dist, key = lambda x:x[1])

## A Naive Cross-Matcher

The process you should follow is:

    Select an object from the BSS catalogue;
    Go through all the objects in SuperCOSMOS and find the closest one to the BSS object;
    If the objects are close enough, record the match;
    Repeat 1-3 for all the other objects in the BSS catalogue.

In step 3, if the closest object isn't within a given distance then it's unlikely that the two objects are actually counterparts, and it's more likely that they just happen to be nearby.

The given distance you choose depends on the uncertainty of the measured object positions in each catalogue. 

In [ ]:
def crossmatch(bss_cat,super_cat,max_dist):
    matches, no_matches, mat = [],[],[]
    for id1,ra1,dec1 in bss_cat:
        min_d = np.inf
        cat_id = id1
        for id2,ra2,dec2 in super_cat:
            d = angular_dist(ra1,dec1,ra2,dec2)
            if d < min_d: 
                min_d = d
                cat_id = id2

        if min_d > max_dist:
            no_matches.append(id1)
        else:
            matches.append((id1,cat_id,min_d))
    return (matches,no_matches)

In [ ]:
bss_cat = import_bss(); super_cat = import_super()
match,no_match = crossmatch(bss_cat,super_cat,40/(60*60))